## Import the libaraies

In [41]:
pip install pyspark

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import *

In [43]:
spark = SparkSession.builder.getOrCreate()
print(spark.version)

3.5.0


In [44]:
df = spark.read.option('header',True).option('inferschema',True).csv('/content/ipl_2022_dataset.csv')
df.show()

+---+-------------------+----------+------------+---------------+---------------+----------+--------------+
|_c0|             Player|Base Price|        TYPE|COST IN ₹ (CR.)|Cost IN $ (000)|2021 Squad|          Team|
+---+-------------------+----------+------------+---------------+---------------+----------+--------------+
|  0|        Rashid Khan|Draft Pick|      BOWLER|           15.0|         1950.0|       SRH|Gujarat Titans|
|  1|      Hardik Pandya|Draft Pick| ALL-ROUNDER|           15.0|         1950.0|        MI|Gujarat Titans|
|  2|    Lockie Ferguson|      2 Cr|      BOWLER|           10.0|         1300.0|       KKR|Gujarat Titans|
|  3|      Rahul Tewatia|   40 Lakh| ALL-ROUNDER|            9.0|         1170.0|        RR|Gujarat Titans|
|  4|       Shubman Gill|Draft Pick|      BATTER|            8.0|         1040.0|       KKR|Gujarat Titans|
|  5|     Mohammad Shami|      2 Cr|      BOWLER|           6.25|          812.5|      PBKS|Gujarat Titans|
|  6|         Yash Dayal|   

# Data Preprocessing

In [45]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Base Price: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- COST IN ₹ (CR.): double (nullable = true)
 |-- Cost IN $ (000): double (nullable = true)
 |-- 2021 Squad: string (nullable = true)
 |-- Team: string (nullable = true)



In [46]:
print('Rows:', df.count())
print('Columns:', len(df.columns))

Rows: 633
Columns: 8


# EDA

In [47]:
df.select('Type').distinct().collect()

[Row(Type='WICKETKEEPER'),
 Row(Type='BOWLER'),
 Row(Type='ALL-ROUNDER'),
 Row(Type='BATTER')]

In [48]:
df.groupBy('Type').count().show()

+------------+-----+
|        Type|count|
+------------+-----+
|WICKETKEEPER|   64|
|      BOWLER|  215|
| ALL-ROUNDER|  242|
|      BATTER|  112|
+------------+-----+



There are total 64 Wicketkeepers, 215 bowlers, 242 All rounders and 112 batsmen in the list.

In [49]:
df.groupBy('2021 Squad').count().show()

+----------+-----+
|2021 Squad|count|
+----------+-----+
|       KKR|   25|
|        DC|   25|
|        RR|   25|
|      NULL|  434|
|        MI|   26|
|       CSK|   25|
|       SRH|   26|
|       RCB|   23|
|      PBKS|   24|
+----------+-----+



There are total 8 teams and 434 unsold players.

In [51]:
#renaming some columns
df = df.withColumnRenamed('Base Price','Base_Price')\
      .withColumnRenamed('COST IN ₹ (CR.)','Cost')\
      .withColumnRenamed('2021 Squad','Old Squad')
df.show(5)

+---+---------------+----------+-----------+----+---------------+---------+--------------+
|_c0|         Player|Base_Price|       TYPE|Cost|Cost IN $ (000)|Old Squad|          Team|
+---+---------------+----------+-----------+----+---------------+---------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|15.0|         1950.0|      SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|15.0|         1950.0|       MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|10.0|         1300.0|      KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER| 9.0|         1170.0|       RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER| 8.0|         1040.0|      KKR|Gujarat Titans|
+---+---------------+----------+-----------+----+---------------+---------+--------------+
only showing top 5 rows



In [52]:
df = df.drop('Cost IN $ (000)')
df.show(5)

+---+---------------+----------+-----------+----+---------+--------------+
|_c0|         Player|Base_Price|       TYPE|Cost|Old Squad|          Team|
+---+---------------+----------+-----------+----+---------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|15.0|      SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|15.0|       MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|10.0|      KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER| 9.0|       RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER| 8.0|      KKR|Gujarat Titans|
+---+---------------+----------+-----------+----+---------+--------------+
only showing top 5 rows



In [53]:
#top 5 players who get paid the most in each category
df.select('Player','Cost').where(df.TYPE == 'BATTER').orderBy('Cost', ascending = False).show(5)

+---------------+-----+
|         Player| Cost|
+---------------+-----+
|   Rohit Sharma| 16.0|
|    Virat Kohli| 15.0|
|Kane Williamson| 14.0|
|   Shreyas Iyer|12.25|
| Mayank Agarwal| 12.0|
+---------------+-----+
only showing top 5 rows



In [54]:
df.select('Player','Cost').where(df.TYPE == 'ALL-ROUNDER').orderBy('Cost', ascending = False).show(5)

+----------------+----+
|          Player|Cost|
+----------------+----+
| Ravindra Jadeja|16.0|
|   Hardik Pandya|15.0|
|   Andre Russell|12.0|
|Liam Livingstone|11.5|
|   Glenn Maxwell|11.0|
+----------------+----+
only showing top 5 rows



In [55]:
df.select('Player','Cost').where(df.TYPE == 'BOWLER').orderBy('Cost', ascending = False).show(5)

+---------------+-----+
|         Player| Cost|
+---------------+-----+
|    Rashid Khan| 15.0|
|  Deepak Chahar| 14.0|
| Jasprit Bumrah| 12.0|
| Shardul Thakur|10.75|
|Lockie Ferguson| 10.0|
+---------------+-----+
only showing top 5 rows



In [56]:
df.select('Player','Cost').where(df.TYPE == 'WICKETKEEPER').orderBy('Cost', ascending = False).show(5)

+------------+-----+
|      Player| Cost|
+------------+-----+
|    KL Rahul| 17.0|
|Rishabh Pant| 16.0|
|Ishan Kishan|15.25|
|Sanju Samson| 14.0|
|    MS Dhoni| 12.0|
+------------+-----+
only showing top 5 rows



In [57]:
#the least paid player in each category
df.select('Player','Cost').filter((df.TYPE == 'BATTER')&(df.Cost.isNotNull())).orderBy('Cost', ascending = True).show(1)

+----------------+----+
|          Player|Cost|
+----------------+----+
|C.Hari Nishaanth| 0.2|
+----------------+----+
only showing top 1 row



In [58]:
df.select('Player','Cost').filter((df.TYPE == 'ALL-ROUNDER')&(df.Cost.isNotNull())).orderBy('Cost', ascending = True).show(1)

+----------------+----+
|          Player|Cost|
+----------------+----+
|Darshan Nalkande| 0.2|
+----------------+----+
only showing top 1 row



In [59]:
df.select('Player','Cost').filter((df.TYPE == 'BOWLER')&(df.Cost.isNotNull())).orderBy('Cost', ascending = True).show(1)

+---------------+----+
|         Player|Cost|
+---------------+----+
|Simarjeet Singh| 0.2|
+---------------+----+
only showing top 1 row



In [60]:
df.select('Player','Cost').filter((df.TYPE == 'WICKETKEEPER')&(df.Cost.isNotNull())).orderBy('Cost', ascending = True).show(1)

+--------------+----+
|        Player|Cost|
+--------------+----+
|Baba Indrajith| 0.2|
+--------------+----+
only showing top 1 row



The highest and lowest paid players include:
Batsman(Rohit Sharma, C.Hari Nishaanth), Bowler (Rashid Khan, Simarjeet Singh), Wicket Keeper(KL Rahul, Baba Indrajith) and All Rounders(Ravindra Jadehja, Darshan Nalkande)

In [61]:
#average price of each category
from pyspark.sql import functions as func
df.groupBy('TYPE').agg(func.round(func.mean('Cost'),2).alias('Average Price')).show()

+------------+-------------+
|        TYPE|Average Price|
+------------+-------------+
|WICKETKEEPER|         5.09|
|      BOWLER|         3.07|
| ALL-ROUNDER|         3.61|
|      BATTER|         4.11|
+------------+-------------+



In [62]:
#top 20 highest bidding players in 2021 squad
df.select('Player','Old Squad','Cost').where(df.Base_Price == 'Retained').orderBy('Cost', ascending = False).show()

+-------------------+---------+----+
|             Player|Old Squad|Cost|
+-------------------+---------+----+
|    Ravindra Jadeja|      CSK|16.0|
|       Rishabh Pant|       DC|16.0|
|       Rohit Sharma|       MI|16.0|
|        Virat Kohli|      RCB|15.0|
|       Sanju Samson|       RR|14.0|
|    Kane Williamson|      SRH|14.0|
|           MS Dhoni|      CSK|12.0|
|      Andre Russell|      KKR|12.0|
|     Mayank Agarwal|     PBKS|12.0|
|     Jasprit Bumrah|       MI|12.0|
|      Glenn Maxwell|      RCB|11.0|
|        Jos Buttler|       RR|10.0|
|         Axar Patel|       DC| 9.0|
|          Moeen Ali|      CSK| 8.0|
|     Venkatesh Iyer|      KKR| 8.0|
|Varun Chakaravarthy|      KKR| 8.0|
|   Suryakumar Yadav|       MI| 8.0|
|       Prithvi Shaw|       DC| 7.5|
|     Mohammed Siraj|      RCB| 7.0|
|      Anrich Nortje|       DC| 6.5|
+-------------------+---------+----+
only showing top 20 rows

